<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

Ce notebook permet de générer des batches de données à partir des fichiers texte de librivox.

In [59]:
import torch
import os
import numpy as np
import re

In [150]:
# constante pour limiter la taille du vocab 
WORD_OCC_THRESHOLD = 3

In [151]:
class Dictionary(object):
    def __init__(self, word_occ_threshold = 0, dico_stats=None):
        self.word2idx = {'<unk>': 0}
        self.idx2word = ['<unk>']
        self.word_occ_threshold = word_occ_threshold
        self.dico_stats = dico_stats
        
    def add_word(self, word):
        word_added = 0
        if self.dico_stats is None:
            if word not in self.word2idx:
                self.idx2word.append(word)
                self.word2idx[word] = len(self.idx2word) - 1
                word_added = 1
        else:
            if word in self.dico_stats.word2stat and \
            self.dico_stats.word2stat[word] > self.word_occ_threshold and \
            word not in self.word2idx:
                
                self.idx2word.append(word)
                self.word2idx[word] = len(self.idx2word) - 1
                word_added = 1
                
        return word_added

    def __len__(self):
        return len(self.idx2word)

In [152]:
class WordStats(object):
    
    def __init__(self, stat_file_path):
        
        self.word2stat = {'<eos>': 1000}
        self.input_file = stat_file_path
        
        with open(self.input_file, 'r') as fh:
            for line in fh:
                occ, wd = line.rstrip().split(' ')
                # substituer le single quote en _
                wd = re.sub("'", "_", wd)
                self.word2stat[wd] = int(occ)
#                 print(wd, self.word2stat[wd])
                
    def __len__(self):
        return len(self.word2stat)    

In [153]:
class Corpus(object):
    
    def __init__(self, path):
        
        self.dictionary_stats = WordStats(os.path.join(path, 'librivox_fr.stats'))
        print(len(self.dictionary_stats))

        self.dictionary = Dictionary(WORD_OCC_THRESHOLD, self.dictionary_stats)
        
        self.train = self.tokenize(os.path.join(path, 'train_librivox_fr_50words_max_15200.txt'))
        self.test = self.tokenize(os.path.join(path, 'test_librivox_fr_50words_max_15200.txt'))
    
    def __len__(self):
        return len(self.test)
    
    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Ajouter les mots au dictionnaire
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    # substituer le single quote en _
                    word = re.sub("'", "_", word)
                    added_bool = self.dictionary.add_word(word)
#                     tokens += added_bool
                tokens += len(words)

        # Tokenizer le contenu du fichier
        with open(path, 'r') as f:
            ids = torch.LongTensor(tokens)
            token = 0
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    # substituer le single quote en _
                    word = re.sub("'", "_", word)
                    
                    if word in self.dictionary.idx2word and self.dictionary.word2idx[word] is not None:
                        ids[token] = self.dictionary.word2idx[word]
                    else:
                        ids[token] = self.dictionary.word2idx['<unk>']
                    token += 1

        return ids
    

In [154]:
data_folder='librivox_fr/data_raw'
corpus = Corpus(data_folder)
len(corpus)

32737


1496

In [155]:
def batchify(data, bsz):
    # Calcul du nb de batches possible en fonction de la taille des batches bsz.
    nbatch = data.size(0) // bsz
    # Retirer le surplus de data
    data = data.narrow(0, 0, nbatch * bsz)
    # Faire le reshape pour obtenir bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data

In [156]:
batch_size=20

train_data = batchify(corpus.train, batch_size)
# val_data = batchify(corpus.valid, batch_size)
test_data = batchify(corpus.test, batch_size)

vocab_size = len(corpus.dictionary)

print(train_data.size())
print(test_data.size())

train_length = train_data.size(0)
test_length  = test_data.size(0)

torch.Size([20542, 20])
torch.Size([74, 20])


In [157]:
torch.save(train_data,'librivox_fr/train_data.pt')
torch.save(test_data,'librivox_fr/test_data.pt')
torch.save(corpus.dictionary.idx2word,'librivox_fr/idx2word.pt')
torch.save(corpus.dictionary.word2idx,'librivox_fr/word2idx.pt')